# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [1]:
!pip install tensorflow==2.2.0rc0

     |████████████████████████████████| 515.9 MB 33 kB/s  eta 0:00:01    |▊                               | 11.2 MB 2.3 MB/s eta 0:03:38     |█▎                              | 21.3 MB 2.3 MB/s eta 0:03:37     |█▊                              | 28.4 MB 6.6 MB/s eta 0:01:15     |█▉                              | 29.4 MB 2.3 MB/s eta 0:03:35     |██▏                             | 35.1 MB 2.2 MB/s eta 0:03:43     |██▊                             | 44.4 MB 1.9 MB/s eta 0:04:06     |███▍                            | 54.5 MB 2.0 MB/s eta 0:03:53     |████▋                           | 74.1 MB 1.9 MB/s eta 0:03:50     |████▋                           | 74.2 MB 1.9 MB/s eta 0:03:50     |████▋                           | 74.8 MB 1.9 MB/s eta 0:03:50     |██████▉                         | 110.8 MB 2.0 MB/s eta 0:03:28     |███████                         | 111.2 MB 2.0 MB/s eta 0:03:27     |███████                         | 113.4 MB 2.2 MB/s eta 0:03:01     |█████████▌                      | 153.2

  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0


In [2]:
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [16]:
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [17]:
model = Sequential()  # Instantiate sequential model
model.add(Dense(512,activation="relu",input_shape=(max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes,activation="softmax")) # Add third layer

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [19]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [21]:
batch_size = 32
epochs = 5
model.fit(x_train,y_train,bacth_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test,verbose=0)

Epoch 1/5
281/281 [==============================] - 2s 7ms/step - loss: 1.3881 - accuracy: 0.6895 - val_loss: 0.9890 - val_accuracy: 0.7707
Epoch 2/5
281/281 [==============================] - 2s 6ms/step - loss: 0.7733 - accuracy: 0.8201 - val_loss: 0.8301 - val_accuracy: 0.8032
Epoch 3/5
281/281 [==============================] - 2s 6ms/step - loss: 0.5491 - accuracy: 0.8687 - val_loss: 0.8250 - val_accuracy: 0.8014
Epoch 4/5
281/281 [==============================] - 2s 7ms/step - loss: 0.4265 - accuracy: 0.8948 - val_loss: 0.8078 - val_accuracy: 0.8063
Epoch 5/5
281/281 [==============================] - 2s 7ms/step - loss: 0.3414 - accuracy: 0.9130 - val_loss: 0.8192 - val_accuracy: 0.8117


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [22]:
score[1]

0.8116651773452759

Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [23]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-07-20 10:28:06--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-07-20 10:28:06 (6.79 MB/s) - ‘rklib.py’ saved [2540/2540]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [ ]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "pont.daniel@gmail.com"
token = "cHw8KTBOCJPKfezm"

submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))